In [ ]:

import pandas as pd
import requests, json

date = '20250201'
stock_no = '2317'

html = requests.get('https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=%s&stockNo=%s' % (date,stock_no))
content = json.loads(html.text)
stock_data = content['data']
col_name = content['fields']

df = pd.DataFrame(data=stock_data, columns=col_name)
df.head(20)

KeyError: 'data'

In [16]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta

def fetch_stock_data(stock_no, start_date, end_date):
    all_data = []

    # 逐月抓取數據
    current_date = datetime.strptime(start_date, "%Y%m%d")
    end_date = datetime.strptime(end_date, "%Y%m%d")

    while current_date <= end_date:
        date_str = current_date.strftime("%Y%m%d")  # 轉換成 YYYYMMDD 格式
        url = f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date_str}&stockNo={stock_no}'

        response = requests.get(url)
        if response.status_code == 200:
            content = json.loads(response.text)
            if 'data' in content and 'fields' in content:
                stock_data = content['data']
                col_name = content['fields']

                df = pd.DataFrame(data=stock_data, columns=col_name)
                all_data.append(df)

        # 移動到下一個月
        next_month = current_date.month % 12 + 1
        next_year = current_date.year + (1 if current_date.month == 12 else 0)
        current_date = datetime(next_year, next_month, 1)

    # 合併所有 DataFrame
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        return None

# 設定查詢範圍
stock_no = '2454'
start_date = '20250131'
end_date = '20250313'

df = fetch_stock_data(stock_no, start_date, end_date)

if df is not None:
    print(df.head(20))  # 顯示前 20 筆資料
    df.to_csv(f"{stock_no}_{start_date}_{end_date}.csv", index=False)  # 存成 CSV 檔案
else:
    print("無法獲取股票數據")


           日期        成交股數            成交金額       開盤價       最高價       最低價  \
0   114/01/02   7,374,055  10,011,026,495  1,395.00  1,400.00  1,335.00   
1   114/01/03   6,413,343   8,684,840,857  1,365.00  1,370.00  1,335.00   
2   114/01/06   7,760,498  10,980,061,110  1,390.00  1,425.00  1,390.00   
3   114/01/07  13,043,149  19,029,056,158  1,445.00  1,500.00  1,415.00   
4   114/01/08  11,341,648  16,799,397,207  1,500.00  1,500.00  1,460.00   
5   114/01/09   7,283,127  10,619,853,655  1,470.00  1,480.00  1,440.00   
6   114/01/10   7,456,378  10,977,244,936  1,460.00  1,495.00  1,445.00   
7   114/01/13  10,467,556  14,785,600,300  1,435.00  1,460.00  1,385.00   
8   114/01/14   5,672,581   7,934,393,535  1,385.00  1,415.00  1,380.00   
9   114/01/15   6,608,704   9,392,555,305  1,390.00  1,445.00  1,390.00   
10  114/01/16   7,526,306  10,959,335,965  1,450.00  1,470.00  1,430.00   
11  114/01/17   5,593,733   8,016,913,060  1,450.00  1,455.00  1,420.00   
12  114/01/20   4,054,089

使用技術分析

台積電

In [2]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta
import pandas_ta as ta  # 技術指標套件

def fetch_stock_data(stock_no, start_date, end_date):
    all_data = []
    current_date = datetime.strptime(start_date, "%Y%m%d")
    end_date = datetime.strptime(end_date, "%Y%m%d")

    while current_date <= end_date:
        date_str = current_date.strftime("%Y%m%d")
        url = f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date_str}&stockNo={stock_no}'

        response = requests.get(url)
        if response.status_code == 200:
            content = json.loads(response.text)
            if 'data' in content and 'fields' in content:
                stock_data = content['data']
                col_name = content['fields']

                df = pd.DataFrame(data=stock_data, columns=col_name)
                all_data.append(df)

        next_month = current_date.month % 12 + 1
        next_year = current_date.year + (1 if current_date.month == 12 else 0)
        current_date = datetime(next_year, next_month, 1)

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        return None

def process_technical_indicators(df):
    # 轉換民國年為西元年，並將日期轉換成 datetime 格式
    def convert_to_ad_date(roc_date):
        # 將 "113/01/02" 轉換成 "2024/01/02"
        year, month, day = roc_date.split('/')
        year = str(int(year) + 1911)  # 民國轉西元
        return f"{year}/{month}/{day}"
    
    df["日期"] = df["日期"].apply(convert_to_ad_date)
    df["日期"] = pd.to_datetime(df["日期"], format="%Y/%m/%d")
    
    # 轉換數據格式
    df["收盤價"] = df["收盤價"].str.replace(",", "").astype(float)
    df["最高價"] = df["最高價"].str.replace(",", "").astype(float)
    df["最低價"] = df["最低價"].str.replace(",", "").astype(float)
    df["開盤價"] = df["開盤價"].str.replace(",", "").astype(float)

    # 計算技術指標
    df["SMA_5"] = ta.sma(df["收盤價"], length=5)  # 5日均線
    df["SMA_20"] = ta.sma(df["收盤價"], length=20)  # 20日均線
    df["SMA_60"] = ta.sma(df["收盤價"], length=60)  # 60日均線
    df["SMA_50"] = ta.sma(df["收盤價"], length=50)  # 50日均線
    df["SMA_200"] = ta.sma(df["收盤價"], length=200)  # 200日均線

    df["EMA_5"] = ta.ema(df["收盤價"], length=5)  # 5日指數均線
    df["EMA_20"] = ta.ema(df["收盤價"], length=20)  # 20日指數均線

    df["RSI"] = ta.rsi(df["收盤價"], length=14)  # RSI 指標
    
    macd_df = ta.macd(df["收盤價"])
    df["MACD"] = macd_df["MACD_12_26_9"]
    df["MACD_signal"] = macd_df["MACDs_12_26_9"]
    df["MACD_hist"] = macd_df["MACDh_12_26_9"]

    

    return df

def calculate_support_resistance(df):
    """
    根據移動平均線來計算支撐與壓力區間
    - 如果收盤價低於某條均線，則該均線可能成為支撐
    - 如果收盤價高於某條均線，則該均線可能成為壓力
    """
    df = df.copy()

    # 計算支撐位（Support Level）
    df["Support_Level"] = df[["SMA_20", "SMA_50", "SMA_200"]].min(axis=1)

    # 計算壓力位（Resistance Level）
    df["Resistance_Level"] = df[["SMA_20", "SMA_50", "SMA_200"]].max(axis=1)

    return df





# 設定查詢範圍
stock_no = '2330'
start_date = '20100101'
end_date = '20241231'

df = fetch_stock_data(stock_no, start_date, end_date)

if df is not None:
    df = process_technical_indicators(df)  # 計算技術指標
    df = calculate_support_resistance(df)  # 加入移動平均線支撐壓力
    #df = calculate_fibonacci(df)  # 如果有斐波那契回撤計算

    print(df.tail(20))  # 顯示最後 20 筆數據
    df.to_csv(f"{stock_no}_{start_date}_{end_date}_technical.csv", index=False)  # 存成 CSV
else:
    print("無法獲取股票數據")



             日期        成交股數            成交金額     開盤價     最高價     最低價     收盤價  \
3661 2024-12-04  35,431,584  37,738,286,825  1065.0  1070.0  1055.0  1070.0   
3662 2024-12-05  41,293,183  44,420,331,699  1070.0  1080.0  1070.0  1075.0   
3663 2024-12-06  32,090,124  34,306,935,479  1075.0  1080.0  1060.0  1065.0   
3664 2024-12-09  30,255,281  32,409,791,931  1070.0  1075.0  1065.0  1075.0   
3665 2024-12-10  32,359,253  34,496,134,808  1070.0  1075.0  1060.0  1065.0   
3666 2024-12-11  26,948,940  28,278,772,503  1045.0  1060.0  1045.0  1045.0   
3667 2024-12-12  26,574,460  28,185,087,817  1060.0  1065.0  1055.0  1060.0   
3668 2024-12-13  24,329,785  25,873,117,992  1060.0  1070.0  1055.0  1065.0   
3669 2024-12-16  43,679,921  47,253,709,874  1085.0  1085.0  1075.0  1085.0   
3670 2024-12-17  40,033,595  43,428,037,809  1090.0  1095.0  1075.0  1075.0   
3671 2024-12-18  47,331,164  51,170,937,776  1075.0  1090.0  1070.0  1085.0   
3672 2024-12-19  46,126,529  49,061,396,399  1065.0 

鴻海

In [3]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta
import pandas_ta as ta  # 技術指標套件
import numpy as np

def fetch_stock_data(stock_no, start_date, end_date):
    all_data = []
    current_date = datetime.strptime(start_date, "%Y%m%d")
    end_date = datetime.strptime(end_date, "%Y%m%d")

    while current_date <= end_date:
        date_str = current_date.strftime("%Y%m%d")
        url = f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date_str}&stockNo={stock_no}'

        response = requests.get(url)
        if response.status_code == 200:
            content = json.loads(response.text)
            if 'data' in content and 'fields' in content:
                stock_data = content['data']
                col_name = content['fields']

                df = pd.DataFrame(data=stock_data, columns=col_name)
                all_data.append(df)

        next_month = current_date.month % 12 + 1
        next_year = current_date.year + (1 if current_date.month == 12 else 0)
        current_date = datetime(next_year, next_month, 1)

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        return None

def process_technical_indicators(df):
    def convert_to_ad_date(roc_date):
        year, month, day = roc_date.split('/')
        year = str(int(year) + 1911)  # 民國轉西元
        return f"{year}/{month}/{day}"
    
    df["日期"] = df["日期"].apply(convert_to_ad_date)
    df["日期"] = pd.to_datetime(df["日期"], format="%Y/%m/%d")
    
    # 轉換數據格式並處理 "--" 遺漏值
    numeric_columns = ["收盤價", "最高價", "最低價", "開盤價"]
    for col in numeric_columns:
        df[col] = df[col].replace('--', np.nan)  # 將 "--" 轉換為 NaN
        df[col] = df[col].str.replace(",", "").astype(float)

    # 可選：填補 NaN 或直接刪除
    df.dropna(inplace=True)  # 移除 NaN 行
    # df.fillna(method='ffill', inplace=True)  # 或用前值填補缺失值

    # 計算技術指標
    df["SMA_5"] = ta.sma(df["收盤價"], length=5)  
    df["SMA_20"] = ta.sma(df["收盤價"], length=20)
    df["SMA_60"] = ta.sma(df["收盤價"], length=60)  # 60日均線  
    df["SMA_50"] = ta.sma(df["收盤價"], length=50)  
    df["SMA_200"] = ta.sma(df["收盤價"], length=200)  

    df["EMA_5"] = ta.ema(df["收盤價"], length=5)  
    df["EMA_20"] = ta.ema(df["收盤價"], length=20)  

    df["RSI"] = ta.rsi(df["收盤價"], length=14)  
    
    macd_df = ta.macd(df["收盤價"])
    df["MACD"] = macd_df["MACD_12_26_9"]
    df["MACD_signal"] = macd_df["MACDs_12_26_9"]
    df["MACD_hist"] = macd_df["MACDh_12_26_9"]

    return df

def calculate_support_resistance(df):
    """
    根據移動平均線來計算支撐與壓力區間
    - 如果收盤價低於某條均線，則該均線可能成為支撐
    - 如果收盤價高於某條均線，則該均線可能成為壓力
    """
    df = df.copy()

    # 計算支撐位（Support Level）
    df["Support_Level"] = df[["SMA_20", "SMA_50", "SMA_200"]].min(axis=1)

    # 計算壓力位（Resistance Level）
    df["Resistance_Level"] = df[["SMA_20", "SMA_50", "SMA_200"]].max(axis=1)

    return df





# 設定查詢範圍
stock_no = '2317'
start_date = '20100101'
end_date = '20241231'

df = fetch_stock_data(stock_no, start_date, end_date)

if df is not None:
    df = process_technical_indicators(df)  # 計算技術指標
    df = calculate_support_resistance(df)  # 加入移動平均線支撐壓力
    #df = calculate_fibonacci(df)  # 如果有斐波那契回撤計算

    print(df.tail(20))  # 顯示最後 20 筆數據
    df.to_csv(f"{stock_no}_{start_date}_{end_date}_technical.csv", index=False)  # 存成 CSV
else:
    print("無法獲取股票數據")



             日期        成交股數            成交金額    開盤價    最高價    最低價    收盤價  \
3639 2024-12-04  50,379,504  10,055,268,944  197.5  201.5  197.0  201.0   
3640 2024-12-05  46,475,440   9,319,676,695  203.0  203.5  199.0  199.0   
3641 2024-12-06  47,526,532   9,465,077,239  199.5  201.5  197.5  198.0   
3642 2024-12-09  41,114,306   8,084,426,101  198.5  199.0  195.0  195.0   
3643 2024-12-10  58,994,150  11,375,383,210  193.0  195.0  192.0  192.0   
3644 2024-12-11  66,246,732  12,596,425,979  190.0  194.0  189.0  189.0   
3645 2024-12-12  50,634,757   9,624,979,364  190.5  193.0  188.5  188.5   
3646 2024-12-13  55,417,050  10,371,852,143  188.5  189.5  185.5  186.5   
3647 2024-12-16  75,228,642  13,834,643,194  188.0  189.5  181.5  182.0   
3648 2024-12-17  55,667,607  10,220,503,792  181.0  186.0  181.0  183.0   
3649 2024-12-18  47,937,386   8,823,563,402  183.0  185.5  182.5  184.5   
3650 2024-12-19  65,029,900  11,756,961,039  181.0  182.0  179.0  181.0   
3651 2024-12-20  56,744,3

聯發科

In [4]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta
import pandas_ta as ta  # 技術指標套件
import numpy as np

def fetch_stock_data(stock_no, start_date, end_date):
    all_data = []
    current_date = datetime.strptime(start_date, "%Y%m%d")
    end_date = datetime.strptime(end_date, "%Y%m%d")

    while current_date <= end_date:
        date_str = current_date.strftime("%Y%m%d")
        url = f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date_str}&stockNo={stock_no}'

        response = requests.get(url)
        if response.status_code == 200:
            content = json.loads(response.text)
            if 'data' in content and 'fields' in content:
                stock_data = content['data']
                col_name = content['fields']

                df = pd.DataFrame(data=stock_data, columns=col_name)
                all_data.append(df)

        next_month = current_date.month % 12 + 1
        next_year = current_date.year + (1 if current_date.month == 12 else 0)
        current_date = datetime(next_year, next_month, 1)

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        return None

def process_technical_indicators(df):
    def convert_to_ad_date(roc_date):
        year, month, day = roc_date.split('/')
        year = str(int(year) + 1911)  # 民國轉西元
        return f"{year}/{month}/{day}"
    
    df["日期"] = df["日期"].apply(convert_to_ad_date)
    df["日期"] = pd.to_datetime(df["日期"], format="%Y/%m/%d")
    
    # 轉換數據格式並處理 "--" 遺漏值
    numeric_columns = ["收盤價", "最高價", "最低價", "開盤價"]
    for col in numeric_columns:
        df[col] = df[col].replace('--', np.nan)  # 將 "--" 轉換為 NaN
        df[col] = df[col].str.replace(",", "").astype(float)

    # 可選：填補 NaN 或直接刪除
    df.dropna(inplace=True)  # 移除 NaN 行
    # df.fillna(method='ffill', inplace=True)  # 或用前值填補缺失值

    # 計算技術指標
    df["SMA_5"] = ta.sma(df["收盤價"], length=5)  
    df["SMA_20"] = ta.sma(df["收盤價"], length=20)
    df["SMA_60"] = ta.sma(df["收盤價"], length=60)  # 60日均線  
    df["SMA_50"] = ta.sma(df["收盤價"], length=50)  
    df["SMA_200"] = ta.sma(df["收盤價"], length=200)  

    df["EMA_5"] = ta.ema(df["收盤價"], length=5)  
    df["EMA_20"] = ta.ema(df["收盤價"], length=20)  

    df["RSI"] = ta.rsi(df["收盤價"], length=14)  
    
    macd_df = ta.macd(df["收盤價"])
    df["MACD"] = macd_df["MACD_12_26_9"]
    df["MACD_signal"] = macd_df["MACDs_12_26_9"]
    df["MACD_hist"] = macd_df["MACDh_12_26_9"]

    return df

def calculate_support_resistance(df):
    """
    根據移動平均線來計算支撐與壓力區間
    - 如果收盤價低於某條均線，則該均線可能成為支撐
    - 如果收盤價高於某條均線，則該均線可能成為壓力
    """
    df = df.copy()

    # 計算支撐位（Support Level）
    df["Support_Level"] = df[["SMA_20", "SMA_50", "SMA_200"]].min(axis=1)

    # 計算壓力位（Resistance Level）
    df["Resistance_Level"] = df[["SMA_20", "SMA_50", "SMA_200"]].max(axis=1)

    return df




# 設定查詢範圍
stock_no = '2454'
start_date = '20100101'
end_date = '20241231'

df = fetch_stock_data(stock_no, start_date, end_date)

if df is not None:
    df = process_technical_indicators(df)  # 計算技術指標
    df = calculate_support_resistance(df)  # 加入移動平均線支撐壓力
    #df = calculate_fibonacci(df)  # 如果有斐波那契回撤計算

    print(df.tail(20))  # 顯示最後 20 筆數據
    df.to_csv(f"{stock_no}_{start_date}_{end_date}_technical.csv", index=False)  # 存成 CSV
else:
    print("無法獲取股票數據")



             日期       成交股數            成交金額     開盤價     最高價     最低價     收盤價  \
3661 2024-12-04  4,844,752   6,364,952,020  1315.0  1325.0  1305.0  1320.0   
3662 2024-12-05  5,559,581   7,367,160,465  1325.0  1335.0  1310.0  1325.0   
3663 2024-12-06  4,364,288   5,747,214,795  1325.0  1340.0  1300.0  1305.0   
3664 2024-12-09  4,592,126   6,045,281,525  1320.0  1325.0  1300.0  1325.0   
3665 2024-12-10  8,230,114  11,015,978,916  1315.0  1360.0  1310.0  1350.0   
3666 2024-12-11  7,633,823  10,191,659,690  1320.0  1355.0  1305.0  1350.0   
3667 2024-12-12  8,447,774  11,722,376,470  1370.0  1400.0  1370.0  1380.0   
3668 2024-12-13  6,733,740   9,392,980,005  1385.0  1410.0  1365.0  1410.0   
3669 2024-12-16  7,521,031  10,583,170,115  1435.0  1440.0  1385.0  1385.0   
3670 2024-12-17  9,362,088  13,364,542,220  1390.0  1445.0  1390.0  1425.0   
3671 2024-12-18  6,198,374   8,798,918,600  1430.0  1430.0  1405.0  1420.0   
3672 2024-12-19  6,719,997   9,496,773,480  1400.0  1430.0  1395